In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
import zipfile

In [3]:
with zipfile.ZipFile('/content/drive/MyDrive/spaceship-titanic.zip') as z:
  print(*z.namelist(), sep='\n')

sample_submission.csv
test.csv
train.csv


In [4]:
with zipfile.ZipFile('/content/drive/MyDrive/spaceship-titanic.zip') as z:
  with z.open('train.csv') as principal:
    train = pd.read_csv(principal)

In [5]:
with zipfile.ZipFile('/content/drive/MyDrive/spaceship-titanic.zip') as t:
  with t.open('test.csv') as trei:
    test = pd.read_csv(trei)

In [6]:
train.shape

(8693, 14)

In [7]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [8]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [9]:
y = train['Transported']

In [10]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [11]:
coluna = ['CryoSleep', 'Destination', 'Cabin', 'Age',
       'VIP']
X = train[coluna]

In [12]:
def transformar_criogenia(gelo):
  if gelo == 'False':
    return 0
  else:
    return 1

In [13]:
def transformar_classe(burgues):
  if burgues == 'False':
    return 0
  else:
    return 1

In [14]:
def transformar_destino(viagem):
  if viagem == 'TRAPPIST-1e':
    return 0
  elif viagem == 'PSO J318.5-22':
    return 1
  else:
    return 2

In [15]:
X['CryoSleep'] = X['CryoSleep'].map(transformar_criogenia)
X['VIP'] = X['VIP'].map(transformar_classe)
X['Destination'] = X['Destination'].map(transformar_destino)

In [16]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [17]:
X

,CryoSleep,Destination,Cabin,Age,VIP
0,1,0,B/0/P,39.0,1
1,1,0,F/0/S,24.0,1
2,1,0,A/0/S,58.0,1
3,1,0,A/0/S,33.0,1
4,1,0,F/1/S,16.0,1
...,...,...,...,...,...
8688,1,2,A/98/P,41.0,1
8689,1,1,G/1499/S,18.0,1
8690,1,0,G/1500/S,26.0,1
8691,1,2,E/608/S,32.0,1


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7)

In [20]:
colunas_numericas = [numeros for numeros in X_train.columns if X_train[numeros].dtype in ['int64', 'float64']]

colunas_categoricos = [categoricos for categoricos in X_train.columns if X_train[categoricos].dtype == 'object' and X_train[categoricos].nunique() < 10]

In [21]:
colunas_numericas

['CryoSleep', 'Destination', 'Age', 'VIP']

In [22]:
colunas_categoricos

[]

In [23]:
colunas_totais = colunas_numericas + colunas_categoricos
X_train = X_train[colunas_totais].copy()
X_valid = X_valid[colunas_totais].copy()
X_teste = test[colunas_totais].copy()

In [24]:
X_train.shape

(6085, 4)

In [25]:
X_valid.shape

(2608, 4)

In [26]:
y_train.shape

(6085,)

In [27]:
y_valid.shape

(2608,)

In [28]:
#imputacao
from sklearn.impute import SimpleImputer

In [29]:
#dados numéricos
imputando = SimpleImputer(strategy='most_frequent')

In [30]:
#chamando o onehotencoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [31]:
from sklearn.pipeline import Pipeline

In [32]:
#dados categóricos
categoricos = Pipeline(steps=[
    ('impt', SimpleImputer(strategy='most_frequent')),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [33]:
#juntando dados numéricos e categóricos
from sklearn.compose import ColumnTransformer

In [34]:
preprocesso = ColumnTransformer(transformers=[
    ('imp', imputando, colunas_numericas),
    ('cate', categoricos, colunas_categoricos)
])

In [35]:
#modelode ml.
from xgboost import XGBClassifier

In [36]:
modelo = XGBClassifier(n_estimators=300)

In [37]:
my_pipeline = Pipeline(steps = [
    ('preprocesso', preprocesso),
    ('model', modelo)
])

In [38]:
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocesso',
                 ColumnTransformer(transformers=[('imp',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['CryoSleep', 'Destination',
                                                   'Age', 'VIP']),
                                                 ('cate',
                                                  Pipeline(steps=[('impt',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  [])])),
                ('model', XGBClassifier(n_estimators=300))])

In [39]:
predicao = my_pipeline.predict(X_valid)
predicao

array([ True,  True, False, ..., False, False, False])

In [40]:
predicao.shape

(2608,)

In [41]:
#testando uma validacao
from sklearn.metrics import mean_absolute_error

In [42]:
y_valid.shape

(2608,)

In [43]:
predicao.shape

(2608,)

In [44]:
y_valid= y_valid.astype(np.int)
predicao = predicao.astype(np.int)

In [45]:
mean_absolute_error(y_valid, predicao)

0.4317484662576687

In [48]:
#precisou ser ajustada a coluna "destination" da parte referente ao treino
#X['CryoSleep'] = X['CryoSleep'].map(transformar_criogenia)
#X['VIP'] = X['VIP'].map(transformar_classe)
X_teste['Destination'] = X_teste['Destination'].map(transformar_destino)

In [46]:
#a

In [49]:
my_pipeline.predict(X_teste)

array([False, False, False, ...,  True,  True, False])